<a href="https://colab.research.google.com/github/Palaeoprot/ModulAAR/blob/main/parameter_optimiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import minimize
from racemization_simulator import run_racemization_simulation

# Gradient descent with early stopping
def gradient_descent_with_early_stopping(X, y, learning_rate=0.01, max_epochs=1000, tolerance=1e-6, patience=10):
    """
    Performs gradient descent with early stopping.

    Parameters:
        X (np.array): Input features.
        y (np.array): Target values.
        learning_rate (float): Learning rate for gradient descent.
        max_epochs (int): Maximum number of epochs.
        tolerance (float): Tolerance for early stopping.
        patience (int): Number of epochs to wait before stopping.

    Returns:
        np.array: Optimized parameters.
    """
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)
    best_theta = np.copy(theta)
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(max_epochs):
        gradients = 2 / n_samples * X.T.dot(X.dot(theta) - y)
        theta -= learning_rate * gradients
        loss = mean_squared_error(y, X.dot(theta))

        if loss < best_loss - tolerance:
            best_loss = loss
            best_theta = np.copy(theta)
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping after {epoch} epochs")
            break

    return best_theta

# Optimization functions
def objective_function(params, real_DL, temp, water_input):
    simulation_results, dl_ratios = run_racemization_simulation(real_DL, params, water_input)
    error = calculate_error(real_DL, dl_ratios, temp)
    return error

def calculate_error(real_DL, dl_ratios, temp):
    # Implement error calculation between real data and simulation results
    error = np.mean((real_DL - dl_ratios) ** 2)  # Example using mean squared error
    return error

def optimize_parameters(real_DL, temperature, initial_params, water_input):
    result = minimize(objective_function, initial_params, args=(real_DL, temperature, water_input),
                      method='Nelder-Mead')
    return result.x

activation_energies = {
    'Asx_DL_BAA': 125000, 'Asx_DL_FAA': 125000,
    'Glx_DL_BAA': 144000, 'Glx_DL_FAA': 144000,
    'Ser_DL_BAA': 126000, 'Ser_DL_FAA': 126000,
    'Ala_DL_BAA': 129000, 'Ala_DL_FAA': 129000,
    'Val_DL_BAA': 118000, 'Val_DL_FAA': 118000,
    'Phe_DL_BAA': 134000, 'Phe_DL_FAA': 134000,
    'Ile_DL_BAA': 120000, 'Ile_DL_FAA': 120000,
    'Hydrolysis': 99000,
    'Gly_loss': 169800, 'Ala_loss': 93200, 'Val_loss': 157000,
    'Leu_loss': 138500, 'Ile_loss': 83100, 'Met_loss': 134100,
    'Phe_loss': 125300, 'Tyr_loss': 69300, 'Ser_loss': 112200,
    'Pro_loss': 49300, 'Lys_loss': 46400, 'His_loss': 110200
}

def adjust_params_for_temperature(params, reference_temp, new_temp):
    R = 8.314  # Gas constant in J/(mol·K)
    adjusted_params = params.copy()

    for param, value in params.items():
        if param in activation_energies:
            Ea = activation_energies[param]
            k_ref = value
            k_new = k_ref * np.exp((Ea/R) * (1/reference_temp - 1/new_temp))
            adjusted_params[param] = k_new

    return adjusted_params

def update_activation_energy(amino_acid, process, value):
    key = f'{amino_acid}_{process}'
    if key in activation_energies:
        activation_energies[key] = value
    else:
        print(f"Warning: Key {key} not found in activation energies dictionary.")
    return activation_energies

# Example usage:
# data = pd.read_csv('/path/to/real_DL_output.csv')
# temperature = 110  # Example temperature
# initial_params = {'k_internal': 0.04, 'k_terminal': 0.01}  # Example initial parameters
# water_input = 100  # Example water input
# optimized_params = optimize_parameters(data, temperature, initial_params, water_input)
# print("Optimized Parameters:", optimized_params)


In [ ]:
import numpy as np
from scipy.optimize import minimize
from racemization_simulator import run_racemization_simulation

def objective_function(params, real_DL, temp, water_input):
    simulation_results, dl_ratios = run_racemization_simulation(real_DL, params, water_input)
    error = calculate_error(real_DL, dl_ratios, temp)
    return error

def calculate_error(real_DL, dl_ratios, temp):
    # Implement error calculation between real data and simulation results
    # ...
    return error

def optimize_parameters(real_DL, temperature, initial_params, water_input):
    result = minimize(objective_function, initial_params, args=(real_DL, temperature, water_input),
                      method='Nelder-Mead')
    return result.x

activation_energies = {
    'Asx_DL_BAA': 125000, 'Asx_DL_FAA': 125000,
    'Glx_DL_BAA': 144000, 'Glx_DL_FAA': 144000,
    'Ser_DL_BAA': 126000, 'Ser_DL_FAA': 126000,
    'Ala_DL_BAA': 129000, 'Ala_DL_FAA': 129000,
    'Val_DL_BAA': 118000, 'Val_DL_FAA': 118000,
    'Phe_DL_BAA': 134000, 'Phe_DL_FAA': 134000,
    'Ile_DL_BAA': 120000, 'Ile_DL_FAA': 120000,
    'Hydrolysis': 99000,
    'Gly_loss': 169800, 'Ala_loss': 93200, 'Val_loss': 157000,
    'Leu_loss': 138500, 'Ile_loss': 83100, 'Met_loss': 134100,
    'Phe_loss': 125300, 'Tyr_loss': 69300, 'Ser_loss': 112200,
    'Pro_loss': 49300, 'Lys_loss': 46400, 'His_loss': 110200
}

def adjust_params_for_temperature(params, reference_temp, new_temp):
    R = 8.314  # Gas constant in J/(mol·K)
    adjusted_params = params.copy()

    for param, value in params.items():
        if param in activation_energies:
            Ea = activation_energies[param]
            k_ref = value
            k_new = k_ref * np.exp((Ea/R) * (1/reference_temp - 1/new_temp))
            adjusted_params[param] = k_new

    return adjusted_params

def update_activation_energy(amino_acid, process, value):
    key = f'{amino_acid}_{process}'
    if key in activation_energies:
        activation_energies[key] = value
    else:
        print(f"Warning: Key {key} not found in activation energies dictionary.")
    return activation_energies